### Q1. MNIST 데이터셋을 Load 하고 DataLoader 를 구현하시오.

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

training_epochs = 15           # training 반복 횟수
batch_size = 100

root = './data'
mnist_train = dset.MNIST(root=root, train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dset.MNIST(root=root, train=False, transform=transforms.ToTensor(), download=True)


In [2]:
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, drop_last=True)

### Q2. 입력 데이터의 형태에 맞도록 linear한 모델을 구성/ 모델 안의 가중치를 초기화
* MNIST 입력의 크기는 28X28
* linear 모델은 입력이 1차원 -> 입력 차원 조정

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# input 은 MNIST data image of shape 28 * 28 = 784, output 은 결과 값이 0 ~ 9 총 10개
linear = nn.Linear(784, 10, bias=True).to(device)
# 가중치 초기화
nn.init.normal_(linear.weight)

Parameter containing:
tensor([[-0.1179,  0.5428, -0.6434,  ..., -0.5106, -1.0171, -2.0350],
        [ 0.7991,  0.4070, -0.2552,  ...,  0.7079,  0.8448,  0.2839],
        [-1.4669,  0.9585, -2.0692,  ...,  1.0633, -1.1607, -0.9036],
        ...,
        [-2.2380, -1.4916,  2.5490,  ...,  0.7198,  1.1922, -1.9419],
        [ 0.6132,  0.5182, -0.3646,  ..., -1.3781, -2.1501,  0.7688],
        [ 0.9000,  1.1057, -1.1974,  ...,  1.5338, -0.7406, -0.7781]],
       requires_grad=True)

### 3. 제시된 loss 함수와 optimizer 를 구현 하세요.
* loss fn - Cross Entropy Loss
* optimizer - SGD

In [4]:
# loss 함수 구현
criterion = nn.CrossEntropyLoss().to(device)
# optimizer 구현, Stochastic Gradient Descent 사용, learning rate = 0.1
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

### Q4. 위의 구현 함수들을 이용해 학습 Loop를 구현하시오.

In [5]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(train_loader)
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28*28)

        outputs = linear(imgs)                 # output 구현
        loss = criterion(outputs, labels)      # outputs와 labels를 비교해서 손실값 구하기

        optimizer.zero_grad()                  # 초기화
        loss.backward()                        # backpropagation 구현
        optimizer.step()                       # optimizer 업데이트

        _, argmax = torch.max(outputs, 1)
        accuracy = (labels == argmax).float().mean()

        avg_cost += loss / total_batch

    print("Epoch: ", "%04d" % (epoch+1), "cost =", "{:.9f}".format(avg_cost), "Accuracy: {:.2f}".format(accuracy.item() * 100))


Epoch:  0001 cost = 2.725812912 Accuracy: 68.00
Epoch:  0002 cost = 1.067796588 Accuracy: 84.00
Epoch:  0003 cost = 0.850940943 Accuracy: 85.00
Epoch:  0004 cost = 0.744687498 Accuracy: 87.00
Epoch:  0005 cost = 0.678898692 Accuracy: 84.00
Epoch:  0006 cost = 0.631606698 Accuracy: 85.00
Epoch:  0007 cost = 0.595635891 Accuracy: 88.00
Epoch:  0008 cost = 0.567253172 Accuracy: 89.00
Epoch:  0009 cost = 0.543816745 Accuracy: 92.00
Epoch:  0010 cost = 0.524150670 Accuracy: 87.00
Epoch:  0011 cost = 0.507252574 Accuracy: 89.00
Epoch:  0012 cost = 0.492583752 Accuracy: 87.00
Epoch:  0013 cost = 0.479427814 Accuracy: 88.00
Epoch:  0014 cost = 0.468342304 Accuracy: 91.00
Epoch:  0015 cost = 0.457778633 Accuracy: 89.00


### Q5. 테스트 데이터를 이용해 테스트를 진행 해보세요.

In [6]:
linear.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (imgs, labels) in enumerate(test_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28 * 28)

        outputs = linear(imgs)

        _, argmax = torch.max(outputs, 1)
        total += imgs.size(0)
        correct += (labels == argmax).sum().item()

    print("Test accuracy for {} images: {:.2f}%".format(total, correct / total * 100))

Test accuracy for 10000 images: 89.23%
